# Buffer1

In [24]:
import pandas as pd

In [25]:
merged_data=pd.read_csv('../data/processed/well_pchip.csv')
gage_to_wells_df = pd.read_csv('../data/processed/wells_with_catchment_info.csv')

In [26]:
import pandas as pd

# Assuming gage_to_wells_df is already loaded
# Convert all column names to lowercase
gage_to_wells_df.columns = gage_to_wells_df.columns.str.lower()

# Drop the specified columns
gage_to_wells_df.drop(columns=['name', 'river', 'state', 'aquifer_na'], inplace=True, errors='ignore')

# Rename the specified columns
gage_to_wells_df.rename(columns={
    'latitude': 'gage_lat',
    'longitude': 'gage_lon',
    'lat_dec': 'well_lat',
    'long_dec': 'well_lon'
}, inplace=True)

# Display the updated DataFrame
print(gage_to_wells_df.head())


        well_id          well_name   well_lat    well_lon     gse  \
0  3.946181e+14  (D-12- 4)16ccb- 1  39.771627 -111.488248  5980.0   
1  3.946341e+14  (D-12- 4)16bcc- 1  39.776071 -111.488248  5960.0   
2  3.946431e+14  (D-12- 4)16bcb- 1  39.778571 -111.487970  5955.0   
3  3.946491e+14  (D-12- 4)17abd- 1  39.780238 -111.494915  5938.0   
4  3.947461e+14  (D-12- 4) 9bab- 1  39.796071 -111.482970  5977.0   

                           geometry  catchment_id     gage_id  \
0   POINT (-111.4882484 39.7716267)   710648999.0  10152000.0   
1   POINT (-111.4882483 39.7760711)   710647014.0  10152000.0   
2  POINT (-111.4879704 39.77857106)   710647014.0  10152000.0   
3  POINT (-111.4949154 39.78023766)   710549798.0  10152000.0   
4  POINT (-111.4829697 39.79607075)   710647014.0  10152000.0   

                             gage_name  
0  SPANISH FORK NEAR LAKE SHORE - UTAH  
1  SPANISH FORK NEAR LAKE SHORE - UTAH  
2  SPANISH FORK NEAR LAKE SHORE - UTAH  
3  SPANISH FORK NEAR LAKE SHOR

In [27]:
well_pchip=merged_data.copy()

In [28]:
import pandas as pd

print("=== Debug: 追踪10126000在merge过程中的变化 ===")

# 在merge之前检查数据
target_gage = 10126000
print("Merge前的数据检查:")
print(f"well_pchip形状: {well_pchip.shape}")
print(f"gage_to_wells_df形状: {gage_to_wells_df.shape}")

# 检查10126000关联的井
wells_for_10126000 = gage_to_wells_df[gage_to_wells_df['gage_id'] == target_gage]
print(f"\n10126000关联的井记录数: {len(wells_for_10126000)}")
if len(wells_for_10126000) > 0:
    target_well_ids = wells_for_10126000['well_id'].unique()
    print(f"10126000关联的井ID: {target_well_ids}")
    
    # 检查这些井在well_pchip中的情况
    wells_in_pchip = well_pchip[well_pchip['well_id'].isin(target_well_ids)]
    print(f"这些井在well_pchip中的记录数: {len(wells_in_pchip)}")
    print(f"这些井的unique well_id: {wells_in_pchip['well_id'].unique() if len(wells_in_pchip) > 0 else 'None'}")
    
    # 检查数据类型
    print(f"\nwell_pchip中well_id的数据类型: {well_pchip['well_id'].dtype}")
    print(f"gage_to_wells_df中well_id的数据类型: {gage_to_wells_df['well_id'].dtype}")
    
    # 检查具体的well_id值
    print(f"\nwell_pchip中的well_id样本: {well_pchip['well_id'].head().tolist()}")
    print(f"gage_to_wells_df中10126000关联的well_id: {target_well_ids.tolist()}")

# 执行原始的merge
print("\n=== 执行merge ===")
filtered_data = pd.merge(
    well_pchip,
    gage_to_wells_df[['gage_id', 'well_id', 'well_lat', 'well_lon']],
    on='well_id',
    how='inner'
)

print(f"Merge后的数据形状: {filtered_data.shape}")
print(f"Merge后的unique gage_id: {sorted(filtered_data['gage_id'].unique())}")
print(f"10126000是否在merge结果中: {target_gage in filtered_data['gage_id'].values}")

# 如果10126000不在结果中，进一步调试
if target_gage not in filtered_data['gage_id'].values:
    print(f"\n=== 10126000丢失原因分析 ===")
    if len(wells_for_10126000) > 0:
        # 逐个检查每个well_id
        for well_id in target_well_ids:
            in_pchip = well_id in well_pchip['well_id'].values
            in_gage_wells = well_id in gage_to_wells_df['well_id'].values
            print(f"Well ID {well_id}: 在well_pchip中={in_pchip}, 在gage_to_wells_df中={in_gage_wells}")
            
            if in_pchip and in_gage_wells:
                # 检查这个well_id是否产生了merge结果
                test_merge = pd.merge(
                    well_pchip[well_pchip['well_id'] == well_id],
                    gage_to_wells_df[gage_to_wells_df['well_id'] == well_id],
                    on='well_id',
                    how='inner'
                )
                print(f"  单独merge这个well_id的结果记录数: {len(test_merge)}")

# Drop the duplicate columns with suffixes
if 'well_lat_x' in filtered_data.columns:
    filtered_data.drop(columns=['well_lat_x', 'well_lon_x'], inplace=True)

# Optionally, rename the remaining columns if needed
if 'well_lat_y' in filtered_data.columns:
    filtered_data.rename(columns={'well_lat_y': 'well_lat', 'well_lon_y': 'well_lon'}, inplace=True)

# Display the new DataFrame
print(f"\n最终处理后的数据形状: {filtered_data.shape}")
print(filtered_data.head())

=== Debug: 追踪10126000在merge过程中的变化 ===
Merge前的数据检查:
well_pchip形状: (26260962, 6)
gage_to_wells_df形状: (2880, 9)

10126000关联的井记录数: 1461
10126000关联的井ID: [4.05443110e+14 4.05452110e+14 4.05459110e+14 ... 4.21441111e+14
 4.21452111e+14 4.21630111e+14]
这些井在well_pchip中的记录数: 3549467
这些井的unique well_id: [410202110560201 410241110523801 410647110503101 411400111000801
 411544110573801 411643110583401 412035111001801 412058111002901
 412248111020301 412315111010301 413124111071601 413124111071602
 413136111094001 413157111094501 413157111484501 413158111485301
 413300111493901 413403111491401 413524111501801 413550111500601
 413559111101901 413609111495701 413616111495701 413620112072301
 413629111543201 413700112073401 413704111543801 413729112100501
 413734111532401 413805111492201 413813112040601 413818111501701
 413822111532901 413827111532501 413837111553101 413840111552601
 413848111545101 413850111551301 413850111570301 413852111514301
 413910111562801 413924111493501 413958111485901 4140021

In [29]:
gage_info = pd.read_csv('../data/raw/streamflow/gsl_nwm_gage.csv')
gage_info

,id,name,River,latitude,longitude,elevation_m,state
0,10011200,WEST FORK BEAR RIVER AT WHITNEY DAM - NR OAKLE...,WEST FORK BEAR RIVER,40.841614,-110.927119,2797,Utah
1,10011500,BEAR RIVER NEAR UTAH-WYOMING STATE LINE,BEAR RIVER,40.965225,-110.853508,2432,Utah
2,10015700,SULPHUR CR.AB.RES.BL.LA CHAPELLE CR.NR EVANSTO...,SULPHUR CREEK,41.129114,-110.806563,2205,Wyoming
3,10015900,SULPHUR CREEK BL RES. - NEAR EVANSTON - WYO.,SULPHUR CREEK,41.156058,-110.835176,2173,Wyoming
4,10016900,BEAR RIVER AT EVANSTON - WY,BEAR RIVER,41.270224,-110.963793,2057,Wyoming
...,...,...,...,...,...,...,...
73,10168500,BIG COTTONWOOD CR NR SALT LAKE CITY UTAH,LITTLE COTTONWOOD CREEK,40.618559,-111.781876,1527,Utah
74,10171000,JORDAN RIVER @ 1700 SOUTH @ SALT LAKE CITY - UT,JORDAN RIVER,40.733557,-111.923270,1294,Utah
75,10172700,VERNON CREEK NEAR VERNON - UT,VERNON CREEK,39.979391,-112.380230,1898,Utah
76,10172860,WARM CREEK NEAR GANDY - UT,WARM CREEK,39.459528,-114.023722,1573,Utah


In [30]:
# Merge gage_info with filtered_data to add gage coordinates
filtered_data = pd.merge(
    filtered_data,
    gage_info[['id', 'latitude', 'longitude']],
    left_on='gage_id',
    right_on='id',
    how='left'
)

# Rename the columns
filtered_data.rename(columns={
    'latitude': 'gage_lat',
    'longitude': 'gage_lon'
}, inplace=True)

# Drop the redundant id column
filtered_data.drop('id', axis=1, inplace=True)


In [31]:
filtered_data.to_csv('../data/processed/after_buffer1.csv', index=False)

In [32]:
# Calculate the number of unique wells
unique_wells_count = filtered_data['well_id'].nunique()

# Display the result
print(f"Number of unique wells: {unique_wells_count}")

unique_gages_count = filtered_data['gage_id'].nunique()
print(f"Number of unique gages: {unique_gages_count}")

Number of unique wells: 1141
Number of unique gages: 8


# buffer2

In [33]:
import pandas as pd
import numpy as np

def filter_and_analyze_wte(filtered_data, merged_df, distance_buffer_meters=30, delta_bins=None):
    # 创建副本避免SettingWithCopyWarning
    filtered_data = filtered_data.copy()
    
    # Convert WTE from feet to meters
    filtered_data['wte_meters'] = filtered_data['wte'] * 0.3048

    # Merge filtered_data with merged_df to get reach_elev
    merged_data = pd.merge(
        filtered_data,
        merged_df[['well_id', 'reach_elev_m']],
        on='well_id',
        how='inner'
    )

    # Calculate the difference between reach elevation and WTE
    merged_data['delta_elev'] = merged_data['reach_elev_m'] - merged_data['wte_meters']

    # Filter out values where the difference is greater than the distance buffer
    filtered_result = merged_data[merged_data['delta_elev'] <= distance_buffer_meters].copy()

    # Define default delta bins if not provided
    if delta_bins is None:
        delta_bins = [-float('inf'), -20, -10, -5, 0, 5, 10, 20, 30, 50, 75, 100, float('inf')]

    # Bin the delta_elev values
    bin_labels = [f"< {delta_bins[1]}"] + \
                 [f"{delta_bins[i]} to {delta_bins[i+1]}" for i in range(1, len(delta_bins)-2)] + \
                 [f">= {delta_bins[-2]}"]
    
    # 修复pandas warning
    filtered_result.loc[:, 'delta_bin'] = pd.cut(filtered_result['delta_elev'], bins=delta_bins, labels=bin_labels)

    # Calculate statistics - 修复FutureWarning
    total_measurements = len(filtered_result)
    dist_stats = filtered_result.groupby('delta_bin', observed=True).size().reset_index(name='count')
    dist_stats['percentage'] = (dist_stats['count'] / total_measurements * 100).round(2)

    # Display statistics
    print(dist_stats)

    return filtered_result, dist_stats

# 高效数据加载 - 只在需要时加载
print("加载数据...")
filtered_data = pd.read_csv('../data/processed/after_buffer1.csv')
merged_df = pd.read_csv('../data/processed/well_reach.csv')

print(f"数据加载完成 - filtered_data: {filtered_data.shape}, merged_df: {merged_df.shape}")

加载数据...
数据加载完成 - filtered_data: (6894716, 9), merged_df: (8752, 9)


In [34]:
filtered_result, dist_stats = filter_and_analyze_wte(filtered_data, merged_df, distance_buffer_meters=30)

    delta_bin   count  percentage
0       < -20  653359       12.92
1  -20 to -10  477643        9.45
2   -10 to -5  512136       10.13
3     -5 to 0  638485       12.63
4      0 to 5  738653       14.61
5     5 to 10  784842       15.52
6    10 to 20  794499       15.71
7    20 to 30  456975        9.04


In [35]:
filtered_result

,well_id,date,wte,gse,gage_id,well_lat,well_lon,gage_lat,gage_lon,wte_meters,reach_elev_m,delta_elev,delta_bin
7178,394643111291401,1971-09-15,5953.300000,5955.0,10152000.0,39.778571,-111.487970,40.150232,-111.726039,1814.565840,1808.5,-6.065840,-10 to -5
7179,394643111291401,1971-09-16,5953.312390,5955.0,10152000.0,39.778571,-111.487970,40.150232,-111.726039,1814.569616,1808.5,-6.069616,-10 to -5
7180,394643111291401,1971-09-17,5953.324794,5955.0,10152000.0,39.778571,-111.487970,40.150232,-111.726039,1814.573397,1808.5,-6.073397,-10 to -5
7181,394643111291401,1971-09-18,5953.337211,5955.0,10152000.0,39.778571,-111.487970,40.150232,-111.726039,1814.577182,1808.5,-6.077182,-10 to -5
7182,394643111291401,1971-09-19,5953.349641,5955.0,10152000.0,39.778571,-111.487970,40.150232,-111.726039,1814.580971,1808.5,-6.080971,-10 to -5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6884353,424553111364801,1968-01-12,5998.253943,6097.0,10126000.0,42.764644,-111.614119,41.576321,-112.100782,1828.267802,1835.5,7.232198,5 to 10
6884354,424553111364801,1968-01-13,5998.286603,6097.0,10126000.0,42.764644,-111.614119,41.576321,-112.100782,1828.277757,1835.5,7.222243,5 to 10
6884355,424553111364801,1968-01-14,5998.320158,6097.0,10126000.0,42.764644,-111.614119,41.576321,-112.100782,1828.287984,1835.5,7.212016,5 to 10
6884356,424553111364801,1968-01-15,5998.354619,6097.0,10126000.0,42.764644,-111.614119,41.576321,-112.100782,1828.298488,1835.5,7.201512,5 to 10


In [36]:
filtered_result.to_csv('../data/processed/after_buffer2_30m.csv', index=False)

## use q dates to filter after buffer2

In [37]:
filtered_data.head()

,well_id,date,wte,gse,gage_id,well_lat,well_lon,gage_lat,gage_lon
0,394618111291501,1970-03-14,5963.000000,5980.0,10152000.0,39.771627,-111.488248,40.150232,-111.726039
1,394618111291501,1970-03-15,5963.000764,5980.0,10152000.0,39.771627,-111.488248,40.150232,-111.726039
2,394618111291501,1970-03-16,5963.001528,5980.0,10152000.0,39.771627,-111.488248,40.150232,-111.726039
3,394618111291501,1970-03-17,5963.002291,5980.0,10152000.0,39.771627,-111.488248,40.150232,-111.726039
4,394618111291501,1970-03-18,5963.003054,5980.0,10152000.0,39.771627,-111.488248,40.150232,-111.726039


In [38]:
filtered_result = filtered_data.copy()

In [39]:
filtered_result.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6894716 entries, 0 to 6894715
Data columns (total 9 columns):
 #   Column    Dtype  
---  ------    -----  
 0   well_id   int64  
 1   date      object 
 2   wte       float64
 3   gse       float64
 4   gage_id   float64
 5   well_lat  float64
 6   well_lon  float64
 7   gage_lat  float64
 8   gage_lon  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 473.4+ MB


In [40]:
compiled_data=pd.read_csv('../data/processed/streamflow/q_bfd_1.csv')

In [41]:
compiled_data.head()

,gage_id,date,q,bfd
0,10015900,1958-04-01,0.0,1.0
1,10015900,1958-04-02,0.0,1.0
2,10015900,1958-04-03,0.0,1.0
3,10015900,1958-04-04,0.0,1.0
4,10015900,1958-04-05,0.0,1.0


In [43]:
import pandas as pd
import numpy as np

print("=== 高效数据类型标准化和merge ===")

# 检查filtered_result中的列
print("检查filtered_result的列:")
print(f"列名: {filtered_result.columns.tolist()}")

# 检查原始数据格式
print(f"\n检查数据类型...")
print(f"filtered_result gage_id类型: {filtered_result['gage_id'].dtype}")
print(f"compiled_data gage_id类型: {compiled_data['gage_id'].dtype}")

# 高效的数据类型标准化 - 直接在原数据上操作，避免复制
print("标准化数据类型...")
filtered_result['gage_id'] = filtered_result['gage_id'].astype('int64')
compiled_data['gage_id'] = compiled_data['gage_id'].astype('int64')

# 检查重合的gage_id
filtered_gages = set(filtered_result['gage_id'])
compiled_gages = set(compiled_data['gage_id'])
overlap = filtered_gages.intersection(compiled_gages)

print(f"数据类型修复后重合的gage_id数量: {len(overlap)}")
print(f"重合的gage_id: {sorted(list(overlap))}")

if len(overlap) > 0:
    print(f"\n日期范围检查:")
    print(f"filtered_result: {filtered_result['date'].min()} - {filtered_result['date'].max()}")
    print(f"compiled_data: {compiled_data['date'].min()} - {compiled_data['date'].max()}")
    
    # 检查是否有wte_meters列，如果没有就创建
    if 'wte_meters' not in filtered_result.columns:
        print("创建wte_meters列...")
        filtered_result['wte_meters'] = filtered_result['wte'] * 0.3048
    
    # 高效merge - 只选择存在的列，减少内存使用
    print("开始高效merge...")
    
    # 动态选择可用的列
    available_cols = ['well_id', 'date', 'wte', 'gse', 'gage_id']
    for col in ['well_lat', 'well_lon', 'gage_lat', 'gage_lon', 'wte_meters']:
        if col in filtered_result.columns:
            available_cols.append(col)
    
    print(f"使用的列: {available_cols}")
    
    q_buffer2_pair = pd.merge(
        filtered_result[available_cols],
        compiled_data[['gage_id', 'date', 'q', 'bfd']],
        on=['gage_id', 'date'],
        how='inner'
    )
    
    print(f"merge成功！记录数: {len(q_buffer2_pair):,}")
    
    if len(q_buffer2_pair) > 0:
        # 验证结果
        print(f"最终数据年份范围: {pd.to_datetime(q_buffer2_pair['date']).dt.year.min()} - {pd.to_datetime(q_buffer2_pair['date']).dt.year.max()}")
        print(f"包含的gage_id: {sorted(q_buffer2_pair['gage_id'].unique())}")
        print(f"最终数据列: {q_buffer2_pair.columns.tolist()}")
        print("\n前5行样本:")
        print(q_buffer2_pair[['well_id', 'date', 'wte', 'gage_id', 'q']].head())
        
        # 保存结果
        print(f"\n保存结果到CSV...")
        q_buffer2_pair.to_csv('../data/processed/q_buffer2_pair_test1.csv', index=False)
        print(f"保存完成，数据形状: {q_buffer2_pair.shape}")
    else:
        print("❌ merge后没有数据")
        q_buffer2_pair = pd.DataFrame()
else:
    print("❌ 修复数据类型后仍然没有重合的gage_id")
    q_buffer2_pair = pd.DataFrame()

# 清理不需要的变量以释放内存
del filtered_gages, compiled_gages
import gc
gc.collect()

print(f"处理完成！")

=== 高效数据类型标准化和merge ===
检查filtered_result的列:
列名: ['well_id', 'date', 'wte', 'gse', 'gage_id', 'well_lat', 'well_lon', 'gage_lat', 'gage_lon']

检查数据类型...
filtered_result gage_id类型: int64
compiled_data gage_id类型: int64
标准化数据类型...
数据类型修复后重合的gage_id数量: 8
重合的gage_id: [10126000, 10141000, 10142000, 10143500, 10152000, 10153100, 10163000, 10168000]

日期范围检查:
filtered_result: 1915-08-01 - 2023-12-06
compiled_data: 1902-10-01 - 2025-01-28
创建wte_meters列...
开始高效merge...
使用的列: ['well_id', 'date', 'wte', 'gse', 'gage_id', 'well_lat', 'well_lon', 'gage_lat', 'gage_lon', 'wte_meters']
merge成功！记录数: 2,348,083
最终数据年份范围: 1932 - 2023
包含的gage_id: [np.int64(10126000), np.int64(10141000), np.int64(10143500), np.int64(10152000), np.int64(10163000), np.int64(10168000)]
最终数据列: ['well_id', 'date', 'wte', 'gse', 'gage_id', 'well_lat', 'well_lon', 'gage_lat', 'gage_lon', 'wte_meters', 'q', 'bfd']

前5行样本:
           well_id        date          wte   gage_id      q
0  394618111291501  1970-03-25  5963.008391  101520

In [44]:
# 高效检查数据匹配情况
print("=== 数据匹配快速检查 ===")

print(f"filtered_result数据: {len(filtered_result):,} 行")
print(f"compiled_data数据: {len(compiled_data):,} 行")

print(f"\nUnique gage IDs:")
print(f"  filtered_result: {len(filtered_result['gage_id'].unique())} 个")
print(f"  compiled_data: {len(compiled_data['gage_id'].unique())} 个")

# 检查重合情况
overlap = set(filtered_result['gage_id']).intersection(set(compiled_data['gage_id']))
print(f"\n重合的gage_id: {len(overlap)} 个")
if len(overlap) > 0:
    print(f"重合ID: {sorted(list(overlap))}")

# 检查日期范围
print(f"\n日期范围:")
print(f"  filtered_result: {filtered_result['date'].min()} - {filtered_result['date'].max()}")
print(f"  compiled_data: {compiled_data['date'].min()} - {compiled_data['date'].max()}")

# 检查数据类型
print(f"\n数据类型:")
print(f"  filtered_result gage_id: {filtered_result['gage_id'].dtype}")
print(f"  compiled_data gage_id: {compiled_data['gage_id'].dtype}")
print(f"  两者日期列类型: {filtered_result['date'].dtype}, {compiled_data['date'].dtype}")

=== 数据匹配快速检查 ===
filtered_result数据: 6,894,716 行
compiled_data数据: 900,056 行

Unique gage IDs:
  filtered_result: 8 个
  compiled_data: 70 个

重合的gage_id: 8 个
重合ID: [10126000, 10141000, 10142000, 10143500, 10152000, 10153100, 10163000, 10168000]

日期范围:
  filtered_result: 1915-08-01 - 2023-12-06
  compiled_data: 1902-10-01 - 2025-01-28

数据类型:
  filtered_result gage_id: int64
  compiled_data gage_id: int64
  两者日期列类型: object, object


In [45]:
q_buffer2_pair.to_csv('../data/processed/q_buffer2_pair_test1.csv', index=False)

In [46]:
print("=== 追踪gage ID 10126000的去向 ===")

# 0. 检查原始的wells_with_catchment_info.csv
print("0. 检查原始catchment数据:")
catchment_raw = pd.read_csv('../data/processed/wells_with_catchment_info.csv')
print(f"原始catchment数据中的unique gage_id: {sorted(catchment_raw['gage_id'].unique()) if 'gage_id' in catchment_raw.columns else 'No gage_id column'}")
target_gage = 10126000
if 'gage_id' in catchment_raw.columns:
    gage_10126000_raw = catchment_raw[catchment_raw['gage_id'] == target_gage]
    print(f"原始数据中gage_id=10126000的记录数: {len(gage_10126000_raw)}")

# 1. 检查well_pchip中是否有相关数据
print("\n1. 检查well_pchip中是否有相关数据:")
if 'well_pchip' not in locals():
    print("重新加载well_pchip数据")
    well_pchip = pd.read_csv('../data/processed/well_pchip.csv')
    
print(f"well_pchip数据形状: {well_pchip.shape}")
print(f"well_pchip中是否包含gage_id列: {'gage_id' in well_pchip.columns}")
if 'gage_id' in well_pchip.columns:
    print(f"well_pchip中的unique gage_id: {well_pchip['gage_id'].unique()}")
else:
    print(f"well_pchip的列: {well_pchip.columns.tolist()}")

# 2. 检查gage_to_wells_df中是否有10126000
print("\n2. 检查处理后的gage_to_wells_df中是否有10126000:")
gage_to_wells_subset = gage_to_wells_df[gage_to_wells_df['gage_id'] == target_gage]
print(f"处理后gage_to_wells_df中gage_id=10126000的记录数: {len(gage_to_wells_subset)}")
print(f"处理后gage_to_wells_df中的unique gage_id: {sorted(gage_to_wells_df['gage_id'].unique())}")

if len(gage_to_wells_subset) > 0:
    print("样本记录:")
    print(gage_to_wells_subset.head())
    print(f"关联的井数量: {gage_to_wells_subset['well_id'].nunique()}")

# 3. 检查数据处理步骤中的丢失
print("\n3. 分析数据处理步骤:")
print("检查在cell 1c7f2eb9c56f7013中的列删除是否影响了数据:")
if len(gage_10126000_raw) > 0 and len(gage_to_wells_subset) == 0:
    print(">> 问题可能在数据清理步骤中！")
    print("原始数据有10126000，但处理后消失了")
    
    # 检查是否在drop columns过程中出了问题
    print("\n检查drop columns的影响:")
    print("删除的列: ['name', 'river', 'state', 'aquifer_na']")
    # 重新处理看看
    test_df = catchment_raw.copy()
    test_df.columns = test_df.columns.str.lower()
    print(f"转小写后的数据中10126000记录数: {len(test_df[test_df['gage_id'] == target_gage])}")
    
    test_df.drop(columns=['name', 'river', 'state', 'aquifer_na'], inplace=True, errors='ignore')
    print(f"删除列后的数据中10126000记录数: {len(test_df[test_df['gage_id'] == target_gage])}")

# 4. 详细检查merge过程
print("\n4. 检查merge逻辑:")
print("Buffer1的merge是基于well_id进行的inner join")
if len(gage_to_wells_subset) > 0:
    target_wells = gage_to_wells_subset['well_id'].unique()
    print(f"10126000关联的井ID: {target_wells}")
    
    wells_in_pchip = well_pchip[well_pchip['well_id'].isin(target_wells)]
    print(f"这些井在well_pchip中的记录数: {len(wells_in_pchip)}")
    if len(wells_in_pchip) == 0:
        print(">> 问题找到了：10126000关联的井在well_pchip中没有数据！")

# 5. 检查after_buffer1的结果
print("\n5. 检查after_buffer1的最终结果:")
print(f"after_buffer1中的unique gage_id: {sorted(filtered_data['gage_id'].unique())}")
print(f"是否包含10126000: {10126000 in filtered_data['gage_id'].values}")

print(f"\nafter_buffer1数据形状: {filtered_data.shape}")

=== 追踪gage ID 10126000的去向 ===
0. 检查原始catchment数据:
原始catchment数据中的unique gage_id: [np.float64(10126000.0), np.float64(10141000.0), np.float64(10142000.0), np.float64(10143500.0), np.float64(10152000.0), np.float64(10153100.0), np.float64(10163000.0), np.float64(10168000.0), np.float64(10172860.0), np.float64(10172952.0)]
原始数据中gage_id=10126000的记录数: 1461

1. 检查well_pchip中是否有相关数据:
well_pchip数据形状: (26260962, 6)
well_pchip中是否包含gage_id列: False
well_pchip的列: ['well_id', 'date', 'wte', 'well_lat', 'well_lon', 'gse']

2. 检查处理后的gage_to_wells_df中是否有10126000:
处理后gage_to_wells_df中gage_id=10126000的记录数: 1461
处理后gage_to_wells_df中的unique gage_id: [np.float64(10126000.0), np.float64(10141000.0), np.float64(10142000.0), np.float64(10143500.0), np.float64(10152000.0), np.float64(10153100.0), np.float64(10163000.0), np.float64(10168000.0), np.float64(10172860.0), np.float64(10172952.0)]
样本记录:
          well_id          well_name   well_lat    well_lon     gse  \
936  4.054431e+14  (A- 2-10)16bdb- 1  40.9118